In [30]:
import numpy as np
import pandas as pd
import xgboost as xgb

from IPython.display import display

df = pd.read_csv('data/train_final.csv', index_col='Id')
df_ult = pd.read_csv('data/test_final.csv', index_col='Id')

df_X = df.drop('Y', axis='columns')
X = df_X.values
y = df['Y'].values

scale_pos_weight = len(y[y == 0])/len(y[y == 1])

dtrain = xgb.DMatrix( X, label=y )

# lgb_train = lgb.Dataset(X, 
#                         label=y,
#                         feature_name=, 
#                         categorical_feature=)

In [35]:
param = {'max_depth': 5, 
         'eta': 0.1, 
         'objective':'binary:logistic', 
         'n_estimators': 750, 
         'booster': 'gbtree', 
         'tree_method': 'auto', 
         'reg_alpha': 0.001, 
         'reg_lambda': 1.0, 
         'eval_metric': 'auc', 
         'nthread': -1}

num_round = 120

def fpreproc(dtrain, dtest, param):
    label = dtrain.get_label()
    ratio = float(np.sum(label == 0)) / np.sum(label == 1)
    param['scale_pos_weight'] = ratio
    return (dtrain, dtest, param)

# do cross validation, for each fold
# the dtrain, dtest, param will be passed into fpreproc
# then the return value of fpreproc will be used to generate
# results of that fold
result = xgb.cv(param, 
                dtrain, 
                num_round, 
                nfold=5, 
                stratified=True, 
                metrics={ 'auc' }, 
                fpreproc=fpreproc)

In [36]:
result

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.846155,0.005035,0.791683,0.010401
1,0.859214,0.003706,0.792887,0.010982
2,0.865602,0.004279,0.793211,0.012422
3,0.873912,0.002276,0.797532,0.014307
4,0.878143,0.002657,0.799226,0.015107
...,...,...,...,...
115,0.982684,0.001536,0.849642,0.011801
116,0.982883,0.001464,0.849531,0.011746
117,0.983084,0.001432,0.849718,0.011600
118,0.983274,0.001403,0.849549,0.011637
